## Imports

In [1]:
import matplotlib.pyplot as plt
from frontier_algorithms.convolutional import ConvolutionalFrontierDetector
from frontier_algorithms.simple_search import SimpleFrontierDetector
from frontier_algorithms.expanding_wavefront import ExpandingWavefront
import numpy as np
from occupancy_grid import OccupancyGrid
from ConcaveHull import ConcaveHull
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

## Gather Ground Truth

For this section, the 'Simple Frontier Detector' is set to just return all frontiers, before performing clustering. This ensures every possible frontier is found, and then is used as the baseline.

In [2]:
map_folder = "maps/"
map_extension = ".tmj"
maps = [("large-field-large-explored", 500, 500), ("large-field-medium-explored", 500, 500), ("medium-field-medium-explored", 250, 250), ("medium-field-large-explored", 250, 250), ("star", 500, 500)]

In [3]:
ground_truth_detector = SimpleFrontierDetector()
convolutional_detectors = [ConvolutionalFrontierDetector(16), ConvolutionalFrontierDetector(32), ConvolutionalFrontierDetector(64), ConvolutionalFrontierDetector(128)]
current_map, x, y = maps[0]
occupancy_grid = OccupancyGrid(f"{map_folder}{current_map}{map_extension}", True)
frontiers_ground = ground_truth_detector.identify_frontiers(occupancy_grid.as_flat(), occupancy_grid.width, occupancy_grid.height, np.array([[250, 250]]), False)
frontiers_test = convolutional_detectors[1].identify_frontiers(occupancy_grid.as_flat(), occupancy_grid.width, occupancy_grid.height, np.array([[250, 250]]))

In [4]:
ch_ground = ConcaveHull()
ch_ground.loadpoints(frontiers_ground)
ch_ground.calculatehull()
boundary_points_ground = np.vstack(ch_ground.boundary.exterior.coords.xy).T.astype(np.int32)

In [5]:
ch_test = ConcaveHull()
ch_test.loadpoints(frontiers_test)
ch_test.calculatehull()
boundary_points_test = np.vstack(ch_test.boundary.exterior.coords.xy).T.astype(np.int32)

## Similarity

In [8]:
distance, path = fastdtw(boundary_points_ground, boundary_points_test, dist=euclidean)
distance

11134.283733285762